## **Desafio - Análise dos resultados dos vídeos de uma playlist**

In [1]:
from configparser import ConfigParser
from googleapiclient.discovery import build
import pandas as pd
import re
import numpy as np

### **Informações sobre a playlist**

Foi obtido o ID da playlist de interesse ao extrair o parâmetro ***list*** da URL. Esse ID foi armazenado na variável `playlist_id`.

In [2]:
# https://www.youtube.com/watch?v=jWPOPjXjiYU&list=PLpbom12S-UaJEDmUaFfWLws317OUKNceE
playlist_id = 'PLpbom12S-UaJEDmUaFfWLws317OUKNceE'

### **Tornando a chave da API disponível para uso**

Foi criado um arquivo .cfg chamado ***credentials.cfg*** para armazenar a chave da API com segurança. Foi utilizada a função `ConfigParser` para ler essa chave e armazená-la na variável `api_key`.

In [3]:
config = ConfigParser()
config.read('credentials.cfg')
api_key = config.get('Google API', 'API_KEY')

### **Criando DataFrame para armazenar os dados**

Foi criado um DataFrame para armazenar as informações que serão extraídas das consultas à API do YouTube. As informações disponíveis foram escolhidas conforme os dados disponíveis na documentação da API e as consultas preliminares feitas a fim de explorar a API.

In [4]:
colunas = ['ID', 'Título', 'Data de Publicação', 'Hora de Publicação', 'Descrição', 'Período de Referência', 'Fundos', 'Duração', 'Dimensão', 'Definição', 'Legendas', 'Conteúdo Licenciado', 'Licença de Uso', 'Tags', 'Privacidade', 'Canal', 'Visualizações', 'Likes', 'Favoritos']
video_infos = pd.DataFrame(columns=colunas)

### **Conexão com a API - Listar vídeos da playlist**

Foi empregada a biblioteca `google-api-python-client` para facilitar a conexão com a API do YouTube. Nessa seção, fez-se o uso da a função `build` para estabelecer a conexão com a API e obter os dados referentes a todos os itens da playlist. Devido ao fato de a API do YouTube retornar, no máximo, 50 itens por vez, foi utilizada a lista `response_list` para armazenar as respostas de cada uma das páginas.

In [5]:
response_list = list()

In [6]:
with build('youtube', 'v3', developerKey=api_key) as service:
    playlist_items = service.playlistItems()
    request = playlist_items.list(part='id, snippet, contentDetails, status', playlistId=playlist_id, maxResults=50)

    while request is not None:
        response = request.execute()
        
        response_list.append(response)
        
        request = playlist_items.list_next(request, response)

As informações mais úteis dessa consulta foram salvas no DataFrame criado previamente.

In [7]:
for page in response_list:
    for video in page['items']:
        video_data = {
            'ID': video['snippet']['resourceId']['videoId'],
            'Título': video['snippet']['title'],
            'Data de Publicação': video['snippet']['publishedAt'],
            'Descrição': video['snippet']['description'],
            'Privacidade': video['status']['privacyStatus'],
            'Canal': video['snippet']['channelTitle'],
        }
        video_data = pd.DataFrame.from_dict([video_data])

        video_infos = pd.concat([video_infos, video_data])

video_infos = video_infos.reset_index(drop=True)

### **Conexão com a API - Encontrar mais informações sobre os vídeos**

Foram definidas as funções `get_biggest_page_number` e `get_videos_ids` que, respectivamente, encontra o maior número de paginação da consulta à API, dado o fato que a maior quantidade de itens por página é 50, e cria uma string com a lista dos 50 IDs dos vídeos da página atual para ser alimentada como parâmetro de busca.

In [8]:
def get_biggest_page_number(video_infos):
    if (len(video_infos)/50 % 1) == 0:
        page = int(len(video_infos)/50)
    else:
        page = int(len(video_infos)/50) + 1
    
    return page


def get_videos_ids(inicio, fim):
    ids = str(list(video_infos['ID'])[inicio:fim])

    ids = re.sub(r'\[', '', ids)
    ids = re.sub(r'\]', '', ids)
    ids = re.sub(r'\s', '', ids)
    ids = re.sub(r"'", '', ids)
    
    return ids

De modo similar à consulta anterior, fez-se o uso da a função `build` para estabelecer a conexão com a API e da lista `response_list` para armazenar as respostas de cada uma das páginas.

In [9]:
response_list = list()

In [10]:
with build('youtube', 'v3', developerKey=api_key) as service:
    max_i = get_biggest_page_number(video_infos)
    
    
    for i in range(max_i-1):
        videos_ids = get_videos_ids(i*50, (i+1)*50)
        
        response = service.videos().list(id=videos_ids, part='id, snippet, contentDetails, player, recordingDetails, statistics, status, topicDetails').execute()
        response_list.append(response)
    

    videos_ids = get_videos_ids((max_i-1)*50, len(video_infos)+1)
    
    response = service.videos().list(id=videos_ids, part='id, snippet, contentDetails, player, recordingDetails, statistics, status, topicDetails').execute()
    response_list.append(response)

As informações mais úteis dessa consulta também foram adicionadas ao DataFrame.

In [11]:
for page in response_list:
    for video in page['items']:
        index = video_infos.loc[video_infos['ID'] == video['id']].index[0]

        video_infos.loc[index, 'Duração'] = video['contentDetails']['duration']
        video_infos.loc[index, 'Dimensão'] = video['contentDetails']['dimension']
        video_infos.loc[index, 'Definição'] = video['contentDetails']['definition']
        video_infos.loc[index, 'Legendas'] = video['contentDetails']['caption']
        video_infos.loc[index, 'Conteúdo Licenciado'] = video['contentDetails']['licensedContent']
        video_infos.loc[index, 'Licença de Uso'] = video['status']['license']
        video_infos.loc[index, 'Visualizações'] = video['statistics']['viewCount']
        video_infos.loc[index, 'Likes'] = video['statistics']['likeCount']
        video_infos.loc[index, 'Favoritos'] = video['statistics']['favoriteCount']
        video_infos.loc[index, 'Comentários'] = video['statistics']['commentCount']
        
        try:
            video_infos.loc[index, 'Tags'] = str(video['snippet']['tags']).replace('[', '').replace(']', '').replace("'", '')
        except:
            pass

### **Removendo vídeos duplicados e privados**

Foi utilizada a função `drop_duplicates` para remover os vídeos duplicados, ou seja, aqueles que possuem o mesmo ID.

In [12]:
video_infos = video_infos.drop_duplicates(subset=['ID'])

Também foram encontrados e removidos do DataFrame os vídeos privados, que contém como ***Private video*** como título.

In [13]:
private_video_indexes = video_infos[video_infos['Título'] == 'Private video'].index

In [14]:
for index in private_video_indexes:
    video_infos = video_infos.drop(index)

### **Organizando os dados de video_infos e hora de publicação e duração**

Foram definidas as funções `format_duration`, `format_date` e `format_time` para formatar as colunas **Duração**, **video_infos de Publicação** e **Hora de Publicação**, respectivamente. Foi utilizado regex para alimentar essas funções e seu retorno foi armazenado nessas colunas.

In [15]:
def format_duration(match):
    if match.group(1):
        hours = int(match.group(1))
    else:
        hours = 0
    
    if match.group(2):
        minutes = int(match.group(2))
    else:
        minutes = 0
    
    if match.group(3):
        seconds = int(match.group(3))
    else:
        seconds = 0
    
    formatted_hours = format(hours, '02d')
    formatted_minutes = format(minutes, '02d')
    formatted_seconds = format(seconds, '02d')

    if formatted_hours == '00':
        return f'{formatted_minutes}:{formatted_seconds}'
    else:
        return f'{formatted_hours}:{formatted_minutes}:{formatted_seconds}'


def format_date(match):
    year = match.group(1)
    month = match.group(2)
    day = match.group(3)

    formatted_date = f'{day}/{month}/{year}'

    return formatted_date


def format_time(match):
    hours = match.group(1)
    minutes = match.group(2)
    seconds = match.group(3)

    formatted_time = f'{hours}:{minutes}:{seconds}'

    return formatted_time

In [16]:
for index, row in video_infos.iterrows():
    duration = row['Duração']

    if re.search(r'PT(\d*)H*(\d+)M(\d*)S*', duration):
        duration = re.sub(r'PT(\d*)H*(\d+)M(\d*)S*', format_duration, duration)
        
        video_infos.loc[index, 'Duração'] = duration
    

    data_hora_publicacao = row['Data de Publicação']

    if re.search(r'(\d{4})-(\d{2})-(\d{2})T(\d{2}):(\d{2}):(\d{2})Z', data_hora_publicacao):
        data_publicacao = re.sub(r'(\d{4})-(\d{2})-(\d{2})T\d{2}:\d{2}:\d{2}Z', format_date, data_hora_publicacao)
        hora_publicacao = re.sub(r'\d{4}-\d{2}-\d{2}T(\d{2}):(\d{2}):(\d{2})Z', format_time, data_hora_publicacao)
        
        video_infos.loc[index, 'Data de Publicação'] = data_publicacao
        video_infos.loc[index, 'Hora de Publicação'] = hora_publicacao

### **Renomeando dados das colunas Dimensão, Definição, Legendas, Conteúdo Licenciado, Licença de Uso e Privacidade**

Foi utilizada a função `replace` para substituir os valores contidos nas colunas Dimensão, Definição, Legendas, Conteúdo Licenciado, Licença de Uso e Privacidade por outros mais semânticos.

In [17]:
print(video_infos['Dimensão'].value_counts(), '\n')
print(video_infos['Definição'].value_counts(), '\n')
print(video_infos['Legendas'].value_counts(), '\n')
print(video_infos['Conteúdo Licenciado'].value_counts(), '\n')
print(video_infos['Licença de Uso'].value_counts(), '\n')
print(video_infos['Privacidade'].value_counts())

2d    301
Name: Dimensão, dtype: int64 

hd    301
Name: Definição, dtype: int64 

false    301
Name: Legendas, dtype: int64 

False    301
Name: Conteúdo Licenciado, dtype: int64 

youtube    301
Name: Licença de Uso, dtype: int64 

public      297
unlisted      4
Name: Privacidade, dtype: int64


In [18]:
legenda = {
    'Dimensão': {
        '2d': '2D',
        '3d': '3D'
    },
    'Definição': {
        'sd': 'SD',
        'hd': 'HD'
    },
    'Legendas': {
        'false': False,
        'true': True
    },
    'Conteúdo Licenciado': {
        'false': False,
        'true': True
    },
    'Licença de Uso': {
        'youtube': 'YouTube',
        'creativeCommon': 'Creative Commons'
    },
    'Privacidade': {
        'public': 'Público',
        'unlisted': 'Não listado'
    }
}

video_infos = video_infos.replace(legenda)

### **Alterando tipos das colunas numéricas**

Os tipos das colunas Legendas, Conteúdo Licenciado, Visualizações, Likes, Favoritos e Comentários foram alterados para fazer jus à realidade.

In [19]:
video_infos = video_infos.astype({
    'Legendas': 'bool',
    'Conteúdo Licenciado': 'bool',
    'Visualizações': 'int',
    'Likes': 'int',
    'Favoritos': 'int',
    'Comentários': 'int'
})

### **Encontrando informações no título (período de referência e fundos)**

Foi utilizado regex e outras operações para encontrar os dados referentes ao Período de Referência e aos Fundos citados em cada vídeo. Esses dados foram adicionados a suas respectivas colunas no DataFrame.

In [20]:
for index, row in video_infos.iterrows():
    titulo = row['Título']


    if re.search(r'(janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro|jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez).*(\d{2,4})', titulo, re.IGNORECASE):
        periodo_referencia = re.findall(r'(janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro|jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez).*(\d{2,4})', titulo, re.IGNORECASE)
        mes_referencia, ano_referencia = periodo_referencia[0]

        substituicoes_mes = {
            'jan': 'Janeiro',
            'fev': 'Fevereiro',
            'mar': 'Março',
            'abr': 'Abril',
            'mai': 'Maio',
            'jun': 'Junho',
            'jul': 'Julho',
            'ago': 'Agosto',
            'set': 'Setembro',
            'out': 'Outubro',
            'nov': 'Novembro',
            'dez': 'Dezembro',
        }
        if mes_referencia.lower() in substituicoes_mes.keys():
            mes_referencia = substituicoes_mes[mes_referencia.lower()]
        
        if len(ano_referencia) == 2:
            ano_referencia = '20' + str(ano_referencia)
        
        periodo_referencia = f'{mes_referencia}/{ano_referencia}'

        video_infos.loc[index, 'Período de Referência'] = periodo_referencia


    fundos = re.findall(r'([^\W\d_]{4}\d{2}|Multimercados|Renda Fixa|RF|Crédito Privado|Ações|CRIs|Fundos Líquidos|Desenvolvimento Imobiliário|Oportunidade)', titulo)
    fundos = str(fundos).replace('[', '').replace(']', '').replace("'", '').replace('RF', 'Renda Fixa')
    
    if fundos == '':
        fundos = None

    video_infos.loc[index, 'Fundos'] = fundos

### **Padronizando valores nulos**

Foi atribuído o valor ***NaN*** a todos os valores nulos do DataFrame, a fim de padronizá-los e facilitar as buscas e a visualização.

In [21]:
video_infos = video_infos.fillna(np.nan)

### **Resultado**

In [22]:
video_infos

,ID,Título,Data de Publicação,Hora de Publicação,Descrição,Período de Referência,Fundos,Duração,Dimensão,Definição,Legendas,Conteúdo Licenciado,Licença de Uso,Tags,Privacidade,Canal,Visualizações,Likes,Favoritos,Comentários
0,dhaE0YL46oQ,Fundo de Infraestrutura KDIF11 - Resumo Mensal...,02/09/2022,18:22:38,Balanço mensal do mês de Agosto de 2022 do fun...,Agosto/2022,KDIF11,06:12,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, fun...",Público,Kinea Investimentos,522,98,0,1
1,gsJhznlLMfM,"Fundos de CRIs (KNCR11, KNIP11, KNHY11, KNSC11...",02/09/2022,01:16:46,Balanço mensal do mês de Agosto de 2022 dos fu...,Agosto/2022,"CRIs, KNCR11, KNIP11, KNHY11, KNSC11, KCRE11",08:36,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimento, inve...",Público,Kinea Investimentos,897,138,0,2
2,yiOWm-SHcBw,Fundo Kinea Crédito Agro - FIAGRO (KNCA11) - R...,02/09/2022,01:06:16,Balanço mensal do mês de Agosto para os fundos...,Agosto/2022,KNCA11,03:03,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimento, inve...",Público,Kinea Investimentos,344,70,0,2
3,QsjAcEGP8D8,Fundos de Ações Kinea - Resumo Mensal ref. Ago...,02/09/2022,00:54:52,Balanço mensal do mês de Agosto de 2022 dos fu...,Agosto/2022,Ações,04:12,2D,HD,False,False,YouTube,"ações, kinea, kinea investimentos, investiment...",Público,Kinea Investimentos,225,30,0,0
5,ECLWwKioHnY,KFOF11 - Resumo Mensal do Kinea FOF FII ref. A...,01/09/2022,23:28:33,Balanço mensal do mês de Agosto de 2022 do fun...,Agosto/2022,KFOF11,03:23,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, fun...",Público,Kinea Investimentos,246,40,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,SnbVnfj21Ec,KNRI11 - Resumo Mensal do Kinea Renda ref. Fev.20,03/03/2020,23:56:49,,Fevereiro/2020,KNRI11,02:10,2D,HD,False,False,YouTube,NaN,Não listado,Kinea Investimentos,420,40,0,0
303,jz-RGae1JK0,KNRI11 - Resumo Mensal do Kinea Renda ref. Jan.20,19/02/2020,13:40:48,,Janeiro/2020,KNRI11,01:53,2D,HD,False,False,YouTube,NaN,Público,Kinea Investimentos,481,44,0,0
304,uOQPYaNQf-I,Prev RF Kinea - Resumo Mensal ref. Jan20,19/02/2020,13:34:26,,Janeiro/2020,Renda Fixa,02:31,2D,HD,False,False,YouTube,NaN,Público,Kinea Investimentos,113,3,0,0
305,kP3SKfI_dBw,Fundos de Ações Kinea - Resumo Mensal Ref. Jan.20,04/02/2020,19:18:08,,Janeiro/2020,Ações,02:12,2D,HD,False,False,YouTube,NaN,Público,Kinea Investimentos,230,4,0,0


In [23]:
video_infos.to_csv('video_infos.csv')

### **Filtros de busca - Período de Referência**

Foi definida a função `search_reference_period` para que buscas possam ser feitas mais facilmente na coluna Período de Referência. Ela recebe um parâmetro que é o período que será buscado, podendo ser apenas um mês, um ano ou a junção de ambos.

In [24]:
def search_reference_period(period):
    indexes = list()

    for index, row in video_infos.iterrows():
        if period in str(row['Período de Referência']):
            indexes.append(index)
    
    return video_infos.loc[indexes]

In [31]:
search_reference_period('Janeiro/2022')

,ID,Título,Data de Publicação,Hora de Publicação,Descrição,Período de Referência,Fundos,Duração,Dimensão,Definição,Legendas,Conteúdo Licenciado,Licença de Uso,Tags,Privacidade,Canal,Visualizações,Likes,Favoritos,Comentários
65,2bQJ-f7gyzE,KFOF11 - Resumo Mensal do Kinea FOF FII ref. J...,03/02/2022,18:14:32,Balanço mensal do mês de Janeiro de 2022 do fu...,Janeiro/2022,KFOF11,02:34,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, fun...",Público,Kinea Investimentos,381,34,0,0
66,bihTR9-A5uM,Fundo Kinea Crédito Agro - FIAGRO (KNCA11) - R...,03/02/2022,15:14:46,Balanço mensal do mês de Janeiro de 2022 do fu...,Janeiro/2022,KNCA11,02:55,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimento, inve...",Público,Kinea Investimentos,58448,94,0,0
67,pkF4hJaIQYU,"Fundos de CRIs (KNCR11, KNIP11, KNHY11 e KNSC1...",02/02/2022,23:03:35,Balanço mensal do mês de Janeiro de 2022 dos f...,Janeiro/2022,"CRIs, KNCR11, KNIP11, KNHY11, KNSC11",04:31,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimento, inve...",Público,Kinea Investimentos,36884,129,0,0
68,KbzhEiq77mY,Fundo de Infraestrutura KDIF11 - Resumo Mensal...,02/02/2022,22:46:14,Balanço mensal do mês de Janeiro de 2022 do fu...,Janeiro/2022,KDIF11,02:53,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, fun...",Público,Kinea Investimentos,55256,81,0,0
69,wlsATBXw0FI,KNRI11 - Resumo Mensal do Kinea Renda ref. Jan...,02/02/2022,22:43:55,Balanço mensal do mês de Janeiro de 2022 do fu...,Janeiro/2022,KNRI11,02:47,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, knr...",Público,Kinea Investimentos,60099,268,0,0
70,i_Bi1BAO1xU,Fundos de Ações Kinea - Resumo Mensal ref. Jan...,02/02/2022,21:35:59,Balanço mensal do mês de Janeiro de 2022 dos f...,Janeiro/2022,Ações,03:01,2D,HD,False,False,YouTube,"ações, kinea, kinea investimentos, investiment...",Público,Kinea Investimentos,36128,17,0,0
71,00nD2_MZ4hc,Crédito Privado Kinea - Resumo Mensal ref. Jan...,31/01/2022,13:10:41,Balanço mensal do mês de Janeiro de 2022 do fu...,Janeiro/2022,Crédito Privado,06:31,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, inv...",Público,Kinea Investimentos,36829,23,0,0
72,BB4BEPOCI6A,Fundos Renda Fixa Kinea - Resumo Mensal ref. J...,31/01/2022,13:07:44,Balanço mensal do mês de Janeiro de 2022 dos f...,Janeiro/2022,Renda Fixa,03:40,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, inv...",Público,Kinea Investimentos,40255,21,0,1
73,b6ts2E1SnpE,Fundos Multimercados Kinea - Resumo Mensal ref...,31/01/2022,13:04:54,Balanço mensal do mês de Janeiro de 2022 dos f...,Janeiro/2022,Multimercados,05:38,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, inv...",Público,Kinea Investimentos,44793,54,0,0


### **Filtros de busca - Fundos**

Foi definida a função `search_funds` para que buscas possam ser feitas mais facilmente na coluna Fundos. Ela recebe um parâmetro que é a sigla fundo que será buscado.

In [26]:
def search_funds(fund):
    indexes = list()

    for index, row in video_infos.iterrows():
        if fund in str(row['Fundos']):
            indexes.append(index)
    
    return video_infos.loc[indexes]

In [27]:
video_infos['Fundos'].value_counts()

Ações                                           33
Renda Fixa                                      31
KFOF11                                          31
Multimercados                                   31
KNRI11                                          30
Crédito Privado                                 30
KDIF11                                          29
CRIs, KNCR11, KNIP11, KNHY11, KNSC11            16
KNHY11                                           9
KNCR11                                           9
KNIP11                                           9
KNCA11                                           8
KINP11                                           7
Desenvolvimento Imobiliário                      6
Oportunidade                                     6
KEVE11                                           5
CRIs, KNCR11, KNIP11, KNHY11, KNSC11, KCRE11     3
KNSC11                                           3
KNCR11, KNIP11, KNHY11                           2
Multimercados, Renda Fixa      

In [32]:
search_funds('Multimercados')

,ID,Título,Data de Publicação,Hora de Publicação,Descrição,Período de Referência,Fundos,Duração,Dimensão,Definição,Legendas,Conteúdo Licenciado,Licença de Uso,Tags,Privacidade,Canal,Visualizações,Likes,Favoritos,Comentários
7,PUCfybCjzhY,Fundos Multimercados Kinea - Resumo Mensal ref...,01/09/2022,20:59:30,Balanço mensal do mês de Agosto de 2022 dos fu...,Agosto/2022,Multimercados,06:46,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, inv...",Público,Kinea Investimentos,791,92,0,2
17,lcJ9vnmlzfE,Fundos Multimercados Kinea - Resumo Mensal ref...,29/07/2022,21:08:21,Balanço mensal do mês de Julho de 2022 dos fun...,Julho/2022,Multimercados,04:21,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, inv...",Público,Kinea Investimentos,24937,75,0,2
25,PIA9_Q0qrxI,Fundos Multimercados Kinea - Resumo Mensal ref...,30/06/2022,21:54:06,Balanço mensal do mês de Junho de 2022 dos fun...,Junho/2022,Multimercados,06:12,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, inv...",Público,Kinea Investimentos,1426,141,0,2
37,autzgYLFBFQ,Fundos Multimercados Kinea - Resumo Mensal ref...,31/05/2022,15:03:35,Balanço mensal do mês de Maio de 2022 dos fund...,Maio/2022,Multimercados,05:02,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, inv...",Público,Kinea Investimentos,32556,98,0,1
46,-SUmsnPU2GU,Fundos Multimercados Kinea - Resumo Mensal ref...,28/04/2022,12:51:05,Balanço mensal do mês de Abril de 2022 dos fun...,Abril/2022,Multimercados,05:18,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, inv...",Público,Kinea Investimentos,37036,90,0,0
54,7xAeLdyTtak,Fundos Multimercados Kinea - Resumo Mensal ref...,31/03/2022,18:14:29,Balanço mensal do mês de Março de 2022 dos fun...,Março/2022,Multimercados,04:02,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, inv...",Público,Kinea Investimentos,45141,72,0,1
63,hgvYPiH5XCU,Fundos Multimercados Kinea - Resumo Mensal ref...,26/02/2022,00:02:50,Balanço mensal do mês de Fevereiro de 2022 dos...,Fevereiro/2022,Multimercados,06:09,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, inv...",Público,Kinea Investimentos,36627,58,0,0
73,b6ts2E1SnpE,Fundos Multimercados Kinea - Resumo Mensal ref...,31/01/2022,13:04:54,Balanço mensal do mês de Janeiro de 2022 dos f...,Janeiro/2022,Multimercados,05:38,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, inv...",Público,Kinea Investimentos,44793,54,0,0
81,sRWtzFSICnE,Fundos Multimercados Kinea - Resumo Mensal ref...,28/12/2021,18:58:30,Balanço mensal do mês de Dezembro de 2021 dos ...,Dezembro/2021,Multimercados,04:55,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, inv...",Público,Kinea Investimentos,768,57,0,0
84,uMYILhrIIe0,Fundos Multimercados Kinea - Resumo Mensal ref...,01/12/2021,17:03:07,Balanço mensal do mês de Novembro de 2021 dos ...,Novembro/2021,Multimercados,04:28,2D,HD,False,False,YouTube,"kinea, kinea investimentos, investimentos, inv...",Público,Kinea Investimentos,34695,66,0,0
